# Developing a CNN for 3x3 Cube Puzzle Metric

## Introduction
This notebook represents the first attempt at developing a Convolutional Neural Network (CNN) as a metric for 3x3 cube puzzles, specifically focusing on the simple solution approach, which involves six values of the cube elements. Traditional methods, like the A* algorithm, often fail to solve these puzzles due to challenges in estimating the distance from a given puzzle state to the solution. 

## Objective
The primary objective of this notebook is to explore the use of a CNN as a distance heuristic for solving 3x3 cube puzzles. 

## Methodology
- **Training the CNN**: The CNN will be trained on a dataset comprising a few hundred thousand puzzles. These puzzles are arranged randomly from the solution state.
- **Goal**: The goal of the CNN is to predict the number of moves required to reach the solution from a given puzzle state.

## Reference
- The A* algorithm used in this project is adapted from Corey Lehmann's notebook available at [Kaggle](https://www.kaggle.com/code/clehmann10/beginner-s-a-star-algorithm-tutorial).


In [7]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation
import matplotlib.pyplot as plt

# local data_dir
data_dir = Path("data")

# Load puzzle info
puzzle_info = pd.read_csv(data_dir / 'puzzle_info.csv', index_col='puzzle_type')
# Parse allowed_moves
puzzle_info['allowed_moves'] = puzzle_info['allowed_moves'].apply(literal_eval)
# puzzle_info_df['total_allowed_moves'] = puzzle_info_df['allowed_moves_dict'].apply(lambda x: len(x.keys()))
puzzle_info['number_moves'] = puzzle_info['allowed_moves'].apply(lambda x: len(x.keys()))

# Load puzzles
puzzles_all = pd.read_csv(data_dir / 'puzzles.csv', index_col='id')
# Parse color states
puzzles_all = puzzles_all.assign(
    initial_state=lambda df: df['initial_state'].str.split(';'),
    solution_state=lambda df: df['solution_state'].str.split(';')
)
# keep only puzzles with puzzle_type in cube_3/3/3
puzzles_all = puzzles_all[puzzles_all['puzzle_type'] == 'cube_3/3/3']

puzzles_all['total_components'] = puzzles_all['solution_state'].apply(len)
puzzles_all['all_unique_components'] = puzzles_all['solution_state'].apply(lambda x: np.unique(x))
puzzles_all['unique_components'] = puzzles_all['all_unique_components'].apply(len)

# ----------------------------------------
# create ditionary of allowed moves
allowed_moves = puzzle_info.loc['cube_3/3/3']['allowed_moves']
# create dictionary of allowed moves converting list to np.array
allowed_moves = {k: np.array(v) for k, v in allowed_moves.items()}
# include inverse moves
move_key = list(allowed_moves.keys())
for i in range(len(allowed_moves)):
    move = allowed_moves[move_key[i]]
    allowed_moves[f"-{move_key[i]}"] = np.argsort(move)

# update move_key
move_key = list(allowed_moves.keys())

# get solution state
solution_state = np.array(puzzles_all.iloc[30]['solution_state'])

# load the sample_submission
ss = pd.read_csv(data_dir / 'sample_submission.csv', index_col='id')
ss['moves'] = ss['moves'].str.split('.')
ss['move_count'] = ss['moves'].apply(len)


## Function Definitions

### 1. `generate_random_state()`
This function is responsible for generating a random state of the puzzle. It operates by applying a random selection of moves to the solution state. The range of moves applied is determined by a specified minimum and maximum. It also takes a set of previous generated states to prevent redundancy. The function then returns three key pieces of information:
   - The resultant state of the puzzle after the moves.
   - The list of moves that were applied.
   - The total number of moves applied.

### 2. `apply_move()`
This function serves as a fundamental operation in manipulating the puzzle state. It is designed to work effectively when the state of the puzzle is represented as a one-dimensional NumPy array. The function takes a move and applies it to the given puzzle state, resulting in a new state that reflects the application of the move.


In [2]:
def apply_move(state, move):
    """
    Apply a move to the cube state.

    Args:
    state (np.array): The current state of the cube.
    move (str): The move to apply.
    allowed_moves (dict): Dictionary of moves and their corresponding permutation functions.

    Returns:
    np.array: New state of the cube after the move.
    """
    return state[move]

import random

def generate_random_state(solution_state, allowed_moves, used_states, min_moves=3, max_moves=20):
    """
    Generate a random cube state by applying a series of random moves to the solution state.
    Checks against a set of used states to avoid duplicates.

    Args:
    solution_state (np.array): The solution state of the cube.
    allowed_moves (dict): Dictionary of moves and their corresponding permutation functions.
    used_states (set): A set of states already used/generated.
    min_moves (int): Minimum number of random moves.
    max_moves (int): Maximum number of random moves.

    Returns:
    np.array: A randomly manipulated state of the cube, or None if a duplicate is found.
    list: The list of moves applied to reach this state, or None if a duplicate is found.
    int: The number of moves applied, or None if a duplicate is found.
    """
    while True:
        num_moves = random.randint(min_moves, max_moves)
        move_sequence = random.choices(list(allowed_moves.keys()), k=num_moves)

        current_state = solution_state.copy()
        for move in move_sequence:
            current_state = apply_move(current_state, allowed_moves[move])

        state_tuple = tuple(current_state.flatten())  # Convert to tuple for set operations
        if state_tuple not in used_states:
            used_states.add(state_tuple)
            return current_state, move_sequence, num_moves
        # If the state is a duplicate, the loop continues to generate a new sequence

## Next generate random moves and transform the puzzle state into a 6 x 3 x 3 x 6 shape for the CNN

In [3]:
from sklearn.preprocessing import OneHotEncoder

# Define all possible labels in the cube states
all_possible_labels = np.array(['A', 'B', 'C', 'D', 'E', 'F']).reshape(-1, 1)

# Fit the OneHotEncoder with all possible labels
encoder = OneHotEncoder()
encoder.fit(all_possible_labels)

def preprocess_cube_state(state, encoder):
    '''
    Preprocess a cube state for input into the CNN
    Args:
    state: 1D array of cube state (3 x 3 x 6 = 54 labels)
    encoder: previously fit OneHotEncoder object
    
    Return: 3D array of one-hot encoded cube state
    '''
    # Reshape the state into a 3x3x6 array (6 faces, each 3x3)
    reshaped_state = state.reshape(6, 3, 3)

    # One-hot encode the labels
    one_hot_encoded = encoder.transform(reshaped_state.reshape(-1, 1)).toarray()

    # Reshape back to 3D structure suitable for CNN (each label is now a one-hot vector)
    one_hot_reshaped = one_hot_encoded.reshape(6, 3, 3, -1)

    return one_hot_reshaped

# ---------------------------------------------------------------------------
training_num = 300_000

# initialize lists to store data
states = []
move_counts = []
used_states = {tuple(solution_state.flatten())}

# generate training_num random states
for i in range(training_num):
    # print progess every 25,000 states
    if i % 25_000 == 0:
        print(f"Generated {i:<8} states out of {training_num}")
        
    random_state, move_sequence, num_moves = generate_random_state(
        solution_state, allowed_moves, used_states,
        min_moves=3, max_moves=20
    )

    # preprocess random_state
    random_state = preprocess_cube_state(random_state, encoder)

    # update used_states
    used_states.add(tuple(random_state.flatten()))

    states.append(random_state.flatten())
    move_counts.append(num_moves)

states = np.array(states)

Generated 0        states out of 300000
Generated 25000    states out of 300000
Generated 50000    states out of 300000
Generated 75000    states out of 300000
Generated 100000   states out of 300000
Generated 125000   states out of 300000
Generated 150000   states out of 300000
Generated 175000   states out of 300000
Generated 200000   states out of 300000
Generated 225000   states out of 300000
Generated 250000   states out of 300000
Generated 275000   states out of 300000


## Creating One-Hot Encoding (OHE) for Puzzle Information

In this section, we'll focus on the preparation of the puzzle data for the Convolutional Neural Network (CNN). The steps involved are as follows:

### Transforming the 1D State Vector
- **Objective**: Convert the one-dimensional (1D) state vector of the puzzle into a more structured format.
- **Method**: We will transform the 1D state vector into a 6x3x3x6 cube. This structured format is more suitable for processing by a CNN, as it captures the spatial relationships inherent in the puzzle. The 6 values of the cube elements with be transformed with the OHE (one-hot encoder)

### Building the Convolutional Neural Network (CNN)
1. **CNN Architecture**: We will design a basic CNN architecture that is capable of processing the transformed puzzle states.
2. **Training the CNN**:
    - **Input**: The CNN will take the recently generated random states (in the 6x3x3x6 format) as input.
    - **Output Goal**: The CNN's objective will be to predict the number of moves (distance) required to reach the solution state from the given puzzle state.
    - **Training Process**: We will detail the training process, including the selection of loss functions, optimizers, and evaluation metrics.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout

def preprocess_cube_state(state, encoder):
    '''
    Preprocess a cube state for input into the CNN
    Args:
    state: 1D array of cube state (3 x 3 x 6 = 54 labels)
    encoder: previously fit OneHotEncoder object
    
    Return: 3D array of one-hot encoded cube state
    '''
    # Reshape the state into a 3x3x6 array (6 faces, each 3x3)
    reshaped_state = state.reshape(6, 3, 3)

    # One-hot encode the labels
    one_hot_encoded = encoder.transform(reshaped_state.reshape(-1, 1)).toarray()

    # Reshape back to 3D structure suitable for CNN (each label is now a one-hot vector)
    one_hot_reshaped = one_hot_encoded.reshape(6, 3, 3, -1)

    return one_hot_reshaped



model = Sequential([
    Conv3D(32, (3, 3, 3), activation='relu', padding='same', input_shape=(6, 3, 3, 6)),
    MaxPooling3D((2, 2, 2)),
    Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)  # Single neuron for regression output
])

Reshape the flattend states and preprocess.

In [5]:
def reshape_data(flattened_data, n_categories):
    """
    Reshape the flattened one-hot encoded data back to its original 3D structure.

    Args:
    flattened_data (np.array): The flattened one-hot encoded data.
    n_categories (int): Number of categories used in one-hot encoding.

    Returns:
    np.array: Reshaped data into its original 3D structure.
    """
    # Calculate the total number of elements in one face
    face_elements = 3 * 3 * n_categories

    # Reshape back to 3D structure (6 faces, 3x3, with one-hot encoded labels)
    reshaped_data = flattened_data.reshape((6, 3, 3, n_categories))

    return reshaped_data

states_reshape = [reshape_data(x, 6) for x in states]
states_reshape = np.array(states_reshape)
move_counts = np.array(move_counts)

In [6]:
model.compile(optimizer='adam',
              loss='mean_squared_error');

history = model.fit(states_reshape, move_counts, epochs=15, validation_split=0.2);

Epoch 1/15
7500/7500 [==============================] - 22s 3ms/step - loss: 15.2661 - val_loss: 11.3529
Epoch 2/15
7500/7500 [==============================] - 22s 3ms/step - loss: 12.8634 - val_loss: 11.4116
Epoch 3/15
7500/7500 [==============================] - 22s 3ms/step - loss: 11.9312 - val_loss: 10.8331
Epoch 4/15
7500/7500 [==============================] - 22s 3ms/step - loss: 11.3986 - val_loss: 10.8302
Epoch 5/15
7500/7500 [==============================] - 22s 3ms/step - loss: 11.0675 - val_loss: 10.7024
Epoch 6/15
7500/7500 [==============================] - 22s 3ms/step - loss: 10.8034 - val_loss: 10.6801
Epoch 7/15
7500/7500 [==============================] - 22s 3ms/step - loss: 10.6414 - val_loss: 10.7676
Epoch 8/15
7500/7500 [==============================] - 22s 3ms/step - loss: 10.5042 - val_loss: 10.8427
Epoch 9/15
7500/7500 [==============================] - 22s 3ms/step - loss: 10.4341 - val_loss: 10.8965
Epoch 10/15
7500/7500 [==============================] 

## Taking a stab at A*
### Added the CNN
- First step looked at single moves. It may slowly converge, but the rate of improvement in the f_score is very slow (code is commented out)
- Second step looked a 3 and 4 random move steps, but this approach **diverges**

In [42]:
import heapq
import time

def get_state_distance(state, encoder):
    """
    Return the distance of the intermediate state from the final state.
    Using the trained CNN model.
    """
    # preprocess state
    state = preprocess_cube_state(state, encoder)
    # reshape state
    pred = model.predict(state.reshape(1, 6, 3, 3, 6), verbose=0)
    
    return pred[0][0]

def evaluate_difference(state1, state2):
    """
    Count the number of positions where the values in two numpy arrays differ.

    Args:
    state1 (np.array): First state, a numpy array.
    state2 (np.array): Second state, a numpy array.

    Returns:
    int: The number of differing positions.
    """
    if state1.shape != state2.shape:
        raise ValueError("States must have the same shape.")

    # Count the number of differing elements
    return np.sum(state1 != state2)

def astar(move_dict, initial_state, final_state, encoder, 
          wildcards, timeout = 180, verbose = False):
    '''
    A* search algorithm. Returns the path of moves to solve the puzzle.
    Inputs:
        move_dict: Dictionary of moves
        initial_state: Initial state of the puzzle
        final_state: Final state of the puzzle
        encoder: OneHotEncoder object
        wildcards: Number of wildcards allowed
        timeout: Maximum time to search
        verbose: Print node number every 1000 nodes
    Outputs:
        path: List of moves to solve the puzzle
        node_counter: Number of nodes visited
        time_delta: Time elapsed
        '''
    
    # Priority queue to store nodes with their f-values (g + h)
    start_time = time.time()
    open_set = []
    node_counter = 1

    move_key = list(move_dict.keys())

    final_state_np = np.array(final_state)
    
    # Use a heap to store nodes with the lowest f-value at the top in open_set
    # heap is a list of tuples (f-value, state, path)
    heapq.heappush(open_set, (0, initial_state, [])) 
    
    # create closed set to store visited nodes
    closed_set = set()

    while open_set:
        # Check for timeout
        time_delta = time.time() - start_time
        if time_delta > timeout:
            print("Timed out.")
            return None, node_counter, time_delta
        
        # Get the node with the lowest f-value from the heap open_set
        current_f, current_state, current_path = heapq.heappop(open_set)
        current_state_np = np.array(current_state)
        
        
        # Check if we've reached the goal and return the path, node_counter, and time_delta
        if evaluate_difference(current_state_np, final_state_np) <= wildcards:
            # We've achieved our goal. Return the move path.
            return current_path, node_counter, time_delta

        # Add the current state to the closed set
        closed_set.add(tuple(current_state))
        ''' Old CODE
        # Add the next possible moves to the open set
        for move_str, move in move_dict.items():
            # apply the move permutation to the current state
            new_state_np = current_state_np[move]
            new_state = list(new_state_np)
            if tuple(new_state) not in closed_set:
                # Add the new state to the open set tuple (f-value, state, path)
                    # f-value = g + h
                        # g is the number of moves taken so far plus 1
                        # h is the heuristic score of the new state using evaluate_score
                    # state is the new state
                    # path is the current path plus the new move

                    f_value = len(current_path) + 1 + get_state_distance(new_state_np, encoder)
                    if verbose and node_counter % 1000 == 0:
                        print(f"node {node_counter} f_value: {current_f}")
                    heapq.heappush(open_set, (f_value, new_state, current_path + [move_str]))
                                    
                                    
                    # Increment the node counter
                    node_counter += 1
                    '''
        
        # create a 20 x 4 list of random moves, 20 lists of 4 random moves
        random_moves = [random.choices(move_key, k=2) for i in range(20)]

        for rand_move in random_moves:
            # apply the move permutation to the current state
            for m_str in rand_move:
                m = allowed_moves[m_str]
                new_state_np = current_state_np[m]
                
            new_state = list(new_state_np)
            if tuple(new_state) not in closed_set:
                # Add the new state to the open set tuple (f-value, state, path)
                    # f-value = g + h
                        # g is the number of moves taken so far plus 1
                        # h is the heuristic score of the new state using evaluate_score
                    # state is the new state
                    # path is the current path plus the new move

                    f_value = len(current_path) + 4 + get_state_distance(new_state_np, encoder)
                    if verbose and node_counter % 1000 == 0:
                        print(f"node {node_counter} f_value: {current_f}")
                    heapq.heappush(open_set, (f_value, new_state, current_path + rand_move))
                                    
                                    
                    # Increment the node counter
                    node_counter += 1

    # end of the while loop
    # If no solutions are found:
    print("Open set completed. No solutions.")
    time_delta = time.time() - start_time
    return None, node_counter, time_delta

In [54]:
for i in range(31, 32):
    # work with puzzle i
    puzzle_id = i
    puzzle = puzzles_all.loc[puzzle_id]

    # # extract initial state and final state as np.array
    # initial_state = np.array(puzzle['initial_state'])
    # final_state = np.array(puzzle['solution_state'])

    initial_state = puzzle['initial_state']
    final_state = puzzle['solution_state']

    path, nodes, time_delta = astar(allowed_moves, initial_state, final_state, encoder, 
            wildcards=0, timeout = 30, verbose = True)
    if path is not None:
        print('SOLVED!')
        print(f"puzzle_id: {puzzle_id}\n\tnodes: {nodes}\n\ttime: {time_delta}\n\tpath: {path}")
    else:
        print('FAIL!!!')
        print(f"puzzle_id: {puzzle_id}\n\tnodes: {nodes}\n\ttime: {time_delta}")

node 1000 f_value: 20.810453414916992
Timed out.
FAIL!!!
puzzle_id: 31
	nodes: 1524
	time: 30.360753059387207


## First attempt to flatten the 3x3x6 cube

In [53]:
state_tmp =  ['A', 'A', 'E', 'C', 'F', 'F', 'C', 'C', 'C', 'D', 'B', 'D', 'D', 'B', 'A', 'B', 'F', 'F', 'A', 'E', 'B', 'D', 'E', 'B', 'D', 'B', 'B', 'A', 'B', 'D', 'F', 'D', 'F', 'F', 'A', 'A', 'E', 'A', 'F', 'C', 'C', 'C', 'C', 'D', 'F', 'C', 'D', 'E', 'E', 'A', 'E', 'B', 'E', 'E']
state_tmp = np.array(state_tmp)
f1 = state_tmp[0:9].reshape(3,3)
f2 = state_tmp[9:18].reshape(3,3)
f3 = state_tmp[18:27].reshape(3,3)
f4 = state_tmp[27:36].reshape(3,3)
f5 = state_tmp[36:45].reshape(3,3)
f6 = state_tmp[45:54].reshape(3,3)

r1 = np.concatenate((f1, f2, f3), axis=1)
r2 = np.concatenate((f4, f5, f6), axis=1)

print(f"f1\n{f1}")
print(f"f2\n{f2}")
print(f"f3\n{f3}")
print(f"r1\n{r1}")
print(f"r2\n{r2}")

flat_cube = np.concatenate((r1, r2), axis=0)
print(f"flat_cube\n{flat_cube}")

f1
[['A' 'A' 'E']
 ['C' 'F' 'F']
 ['C' 'C' 'C']]
f2
[['D' 'B' 'D']
 ['D' 'B' 'A']
 ['B' 'F' 'F']]
f3
[['A' 'E' 'B']
 ['D' 'E' 'B']
 ['D' 'B' 'B']]
r1
[['A' 'A' 'E' 'D' 'B' 'D' 'A' 'E' 'B']
 ['C' 'F' 'F' 'D' 'B' 'A' 'D' 'E' 'B']
 ['C' 'C' 'C' 'B' 'F' 'F' 'D' 'B' 'B']]
r2
[['A' 'B' 'D' 'E' 'A' 'F' 'C' 'D' 'E']
 ['F' 'D' 'F' 'C' 'C' 'C' 'E' 'A' 'E']
 ['F' 'A' 'A' 'C' 'D' 'F' 'B' 'E' 'E']]
flat_cube
[['A' 'A' 'E' 'D' 'B' 'D' 'A' 'E' 'B']
 ['C' 'F' 'F' 'D' 'B' 'A' 'D' 'E' 'B']
 ['C' 'C' 'C' 'B' 'F' 'F' 'D' 'B' 'B']
 ['A' 'B' 'D' 'E' 'A' 'F' 'C' 'D' 'E']
 ['F' 'D' 'F' 'C' 'C' 'C' 'E' 'A' 'E']
 ['F' 'A' 'A' 'C' 'D' 'F' 'B' 'E' 'E']]


## More efficient transformation

In [59]:
# Linear representation of the Rubik's cube
state_tmp = ['A', 'A', 'E', 'C', 'F', 'F', 'C', 'C', 'C', 'D', 'B', 'D', 'D', 'B', 'A', 'B', 'F', 'F', 'A', 'E', 'B', 'D', 'E', 'B', 'D', 'B', 'B', 'A', 'B', 'D', 'F', 'D', 'F', 'F', 'A', 'A', 'E', 'A', 'F', 'C', 'C', 'C', 'C', 'D', 'F', 'C', 'D', 'E', 'E', 'A', 'E', 'B', 'E', 'E']
state_tmp = np.array(state_tmp)

# Reshape each 9-element block into a 3x3 face
faces = [state_tmp[i:i+9].reshape(3,3) for i in range(0, 54, 9)]

# Concatenate the first three and the last three faces
r1 = np.concatenate(faces[0:3], axis=1)
r2 = np.concatenate(faces[3:6], axis=1)

# Final 6x9 matrix representing the Rubik's cube
rubiks_matrix = np.concatenate((r1, r2), axis=0)

print(rubiks_matrix)

# RGB values for the colors of a Rubik's cube
color_map = {
    'A': [255, 255, 255],  # White
    'B': [255, 0, 0],      # Red
    'C': [0, 0, 255],      # Blue
    'D': [255, 165, 0],    # Orange
    'E': [0, 255, 0],      # Green
    'F': [255, 255, 0]     # Yellow
}

# Transform the rubiks_matrix into a 3D matrix with RGB values
rubiks_rgb_matrix = np.array([[color_map[element] for element in row] for row in rubiks_matrix])
rubiks_rgb_matrix

print(rubiks_rgb_matrix)
print(rubiks_rgb_matrix.shape)

[['A' 'A' 'E' 'D' 'B' 'D' 'A' 'E' 'B']
 ['C' 'F' 'F' 'D' 'B' 'A' 'D' 'E' 'B']
 ['C' 'C' 'C' 'B' 'F' 'F' 'D' 'B' 'B']
 ['A' 'B' 'D' 'E' 'A' 'F' 'C' 'D' 'E']
 ['F' 'D' 'F' 'C' 'C' 'C' 'E' 'A' 'E']
 ['F' 'A' 'A' 'C' 'D' 'F' 'B' 'E' 'E']]
[[[255 255 255]
  [255 255 255]
  [  0 255   0]
  [255 165   0]
  [255   0   0]
  [255 165   0]
  [255 255 255]
  [  0 255   0]
  [255   0   0]]

 [[  0   0 255]
  [255 255   0]
  [255 255   0]
  [255 165   0]
  [255   0   0]
  [255 255 255]
  [255 165   0]
  [  0 255   0]
  [255   0   0]]

 [[  0   0 255]
  [  0   0 255]
  [  0   0 255]
  [255   0   0]
  [255 255   0]
  [255 255   0]
  [255 165   0]
  [255   0   0]
  [255   0   0]]

 [[255 255 255]
  [255   0   0]
  [255 165   0]
  [  0 255   0]
  [255 255 255]
  [255 255   0]
  [  0   0 255]
  [255 165   0]
  [  0 255   0]]

 [[255 255   0]
  [255 165   0]
  [255 255   0]
  [  0   0 255]
  [  0   0 255]
  [  0   0 255]
  [  0 255   0]
  [255 255 255]
  [  0 255   0]]

 [[255 255   0]
  [255 255 255]
  [

Yes, the transformed 3D matrix with RGB values representing the state of a Rubik's Cube can be used as input data for a Convolutional Neural Network (CNN) to build a model that estimates how far a given puzzle state is from the solved state. Here's an overview of how this could be approached:

1. **Data Preparation**: 
    - Your dataset should consist of various Rubik's Cube states, each represented as a 3D RGB matrix like the one you've created.
    - For each state, you would need a corresponding label indicating its distance from the solved state. This could be a numerical value representing the minimum number of moves required to solve the puzzle from that state.

2. **CNN Model Design**: 
    - Design a CNN that can process the 3D RGB matrix. The network would likely include several convolutional layers to extract features from the cube's state.
    - After convolutional layers, you might use fully connected layers that output a prediction of the distance from the solved state.

3. **Training and Validation**:
    - You would need to train this model on a large and varied dataset of Rubik's Cube states and their distances from the solved state.
    - Regular validation during training would be necessary to adjust hyperparameters and prevent overfitting.

4. **Model Evaluation**:
    - After training, evaluate the model's performance on a test set of cube states that were not seen during training.
    - Key performance metrics could include accuracy, mean squared error, or other relevant metrics depending on how you define the distance from the solved state.

5. **Challenges**:
    - One of the main challenges would be generating or acquiring a dataset with accurately labeled distances from the solved state.
    - The model might also need to be quite complex to accurately predict the distance, given the complexity of the Rubik's Cube's state space.

6. **Post-Processing**:
    - The output from the CNN would give you an estimate of how far a given puzzle state is from being solved. This could potentially be used to guide a solving algorithm or for educational purposes.

Remember, while CNNs are powerful for image and pattern recognition tasks, predicting the specific distance from the solved state in a Rubik's Cube is a highly complex problem, and the success of such a model would depend greatly on the quality and diversity of the training data, as well as the architecture of the neural network.


In [63]:
puzzle_id = 30

puzzle = puzzles_all.loc[puzzle_id]
initial_state = puzzle['initial_state']
final_state = puzzle['solution_state']

print(initial_state)

# preprocess initial_state
initial_state = preprocess_cube_state(np.array(initial_state), encoder)

# predict distance
pred = model.predict(initial_state.reshape(1, 6, 3, 3, 6))
# get distance
distance = int(pred[0][0])
print(distance)

temp_state = np.array(final_state)
temp_state = temp_state[allowed_moves['f0']]
temp_state = temp_state[allowed_moves['r1']]
temp_state = temp_state[allowed_moves['d0']]
temp_state = temp_state[allowed_moves['r0']]
temp_state = preprocess_cube_state(temp_state, encoder)
pred = model.predict(temp_state.reshape(1, 6, 3, 3, 6))
print(pred[0][0])

['A', 'A', 'E', 'C', 'F', 'F', 'C', 'C', 'C', 'D', 'B', 'D', 'D', 'B', 'A', 'B', 'F', 'F', 'A', 'E', 'B', 'D', 'E', 'B', 'D', 'B', 'B', 'A', 'B', 'D', 'F', 'D', 'F', 'F', 'A', 'A', 'E', 'A', 'F', 'C', 'C', 'C', 'C', 'D', 'F', 'C', 'D', 'E', 'E', 'A', 'E', 'B', 'E', 'E']
1/1 [==============================] - 0s 11ms/step


13
1/1 [==============================] - 0s 8ms/step
6.9100885


In [31]:
print(move_key)
random_moves = [random.choices(move_key, k=4) for i in range(20)]
print(random_moves)
for seq in random_moves:
    print(seq)
    for m in seq:
        print(f"\tmove: {m}")

['f0', 'f1', 'f2', 'r0', 'r1', 'r2', 'd0', 'd1', 'd2', '-f0', '-f1', '-f2', '-r0', '-r1', '-r2', '-d0', '-d1', '-d2']
[['r2', '-f0', 'r1', 'd1'], ['r1', 'r0', '-f0', 'r0'], ['f0', '-d1', '-f1', '-f0'], ['-f1', 'f1', '-d0', '-f1'], ['f2', '-f2', 'r0', '-r0'], ['r0', 'f1', 'r0', '-d1'], ['r2', 'f2', '-f2', '-d0'], ['f1', 'r0', 'r2', 'r0'], ['-f1', '-d2', '-r1', 'r0'], ['-d1', '-d1', '-r2', '-f0'], ['-r1', '-d1', '-r0', 'r0'], ['-f0', 'r0', '-r2', '-f1'], ['f0', '-d2', '-d1', '-f0'], ['-d2', '-f2', 'r2', '-r2'], ['-f1', '-r2', '-r2', '-r0'], ['f0', '-d0', '-r1', 'f0'], ['-r2', '-f0', '-f2', 'f2'], ['-f0', '-f2', '-d2', '-d1'], ['-r0', 'f1', 'd2', '-d0'], ['-d0', 'r0', '-r2', 'r0']]
['r2', '-f0', 'r1', 'd1']
	move: r2
	move: -f0
	move: r1
	move: d1
['r1', 'r0', '-f0', 'r0']
	move: r1
	move: r0
	move: -f0
	move: r0
['f0', '-d1', '-f1', '-f0']
	move: f0
	move: -d1
	move: -f1
	move: -f0
['-f1', 'f1', '-d0', '-f1']
	move: -f1
	move: f1
	move: -d0
	move: -f1
['f2', '-f2', 'r0', '-r0']
	move: f2

In [43]:

# for each puzzle in puzzles_temp check if solution_state == solution_state
puzzle_ids = []
for i in range(len(puzzles_all)):
    if puzzles_all.iloc[i]['solution_state'] == solution_state:
        puzzle_ids.append(i)
print(np.array(puzzle_ids))

puzzles_all.iloc[30]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


puzzle_type                                                     cube_3/3/3
solution_state           [A, A, A, A, A, A, A, A, A, B, B, B, B, B, B, ...
initial_state            [A, A, E, C, F, F, C, C, C, D, B, D, D, B, A, ...
num_wildcards                                                            0
total_components                                                        54
all_unique_components                                   [A, B, C, D, E, F]
unique_components                                                        6
Name: 30, dtype: object

In [46]:
for i in puzzle_ids:
    print(f"puzzle {i:>4}: number of moves: {ss.iloc[i]['move_count']}")

puzzle    0: number of moves: 2
puzzle    1: number of moves: 63
puzzle    2: number of moves: 62
puzzle    3: number of moves: 92
puzzle    4: number of moves: 70
puzzle    5: number of moves: 54
puzzle    6: number of moves: 68
puzzle    7: number of moves: 83
puzzle    8: number of moves: 98
puzzle    9: number of moves: 76
puzzle   10: number of moves: 66
puzzle   11: number of moves: 63
puzzle   12: number of moves: 72
puzzle   13: number of moves: 131
puzzle   14: number of moves: 96
puzzle   15: number of moves: 68
puzzle   16: number of moves: 63
puzzle   17: number of moves: 62
puzzle   18: number of moves: 89
puzzle   19: number of moves: 82
puzzle   20: number of moves: 112
puzzle   21: number of moves: 96
puzzle   22: number of moves: 63
puzzle   23: number of moves: 53
puzzle   24: number of moves: 99
puzzle   25: number of moves: 61
puzzle   26: number of moves: 93
puzzle   27: number of moves: 73
puzzle   28: number of moves: 83
puzzle   29: number of moves: 82
puzzle   

IndexError: single positional indexer is out-of-bounds